<div style='float: right'><img src='pic/numberlink.png'/></div>
## <div id='numberlink' />ナンバーリンク

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = np.array([list(s) for s in """\
1.4....
2....3.
...2...
.......
.......
..5.4..
1.3...5""".split()])
data[data == '.'] = '0'
data = data.astype(int)

### 問題
* 各マスには2つ以上の線が入ってはいけません
* 異なる線同士が交わってはいけません

### 変数
* vr:どのタイプのラインに含まれるか (1)
* vh:水平に出るかどうか (2)
* vv:垂直に出るかどうか (3)

### 制約
* 始点と終点が指定のラインに含まれること (4)
* 始点と終点は、1本だけ出ること (5)
* 始点終点以外の各マスに接続されるのは、0本か2本 (6)
* 接続したら、両端のタイプは同じになること (7)

In [ ]:
mx = data.max()
m = LpProblem()
vr = addvars(*data.shape) # (1)
vh = addbinvars(data.shape[0], data.shape[1]-1) # (2)
vv = addbinvars(data.shape[0]-1, data.shape[1]) # (3)
def dirs(i, j):
    return ([vh[i][j - k] for k in range(2) if 0 <= j-k < data.shape[1] - 1]
          + [vv[i - k][j] for k in range(2) if 0 <= i-k < data.shape[0] - 1])
for i, j in product(range(data.shape[0]), range(data.shape[1])):
    s = dirs(i, j)
    if data[i][j]:
        m += vr[i][j] == data[i][j] # (4)
        m += lpSum(s) == 1 # (5)
    else:
        m += lpSum(s) == 2 * addbinvar() # (6)
    if i < data.shape[0] - 1:
        m += vr[i][j] <= vr[i + 1][j] + mx * (1 - vv[i][j]) # (7)
        m += vr[i + 1][j] <= vr[i][j] + mx * (1 - vv[i][j]) # (7)
    if j < data.shape[1] - 1:
        m += vr[i][j] <= vr[i][j + 1] + mx * (1 - vh[i][j]) # (7)
        m += vr[i][j + 1] <= vr[i][j] + mx * (1 - vh[i][j]) # (7)
%time m.solve()
np.vectorize(value)(vr).astype(int).tolist()